In [10]:
# !pip uninstall tensorflow
# !pip install tensorflow

In [11]:
# !pip install tqdm

In [12]:
# !pip show tqdm

In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import tensorflow as tf

#tf.disable_v2_behavior()
import cv2 as cv
import nibabel as nib
import pydicom as pyd
import gdcm

In [2]:
import PIL

In [3]:
#import 

# Link data


In [4]:
#datalink = "D:/dataset/pcxr/images/"
TRAIN = 'train/'
TEST = 'test/'
LABELS = 'labels/'

In [5]:
import os
files = os.listdir(LABELS)

# Filter files with the ".csv" extension
csv_files = [file for file in files if file.endswith('.csv')]

# Print the list of CSV files
print(csv_files)

['annotations_test.csv', 'annotations_train.csv', 'image_labels_test.csv', 'image_labels_train.csv']


In [9]:
train_df = pd.read_csv(LABELS + 'annotations_train.csv')
train_df.head()

,image_id,rad_ID,class_name,x_min,y_min,x_max,y_max,class_id
0,03d56a9709a81b2e9d6afbc6647c8730,R3,Boot-shaped heart,378.760010,401.416645,835.143248,661.678437,0
1,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,247.196368,412.000819,441.763442,830.320028,1
2,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,613.631024,364.980443,902.238850,827.077243,1
3,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,1008.515903,665.439667,1324.430364,1265.465119,1
4,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,453.015307,635.756429,724.404911,1242.142575,1


In [10]:
dim = 256

In [11]:
train_df['image_path'] = [(TRAIN + train_df['image_id'][i] + '.dicom') for i in range(len(train_df))]
train_df.head()

,image_id,rad_ID,class_name,x_min,y_min,x_max,y_max,class_id,image_path
0,03d56a9709a81b2e9d6afbc6647c8730,R3,Boot-shaped heart,378.760010,401.416645,835.143248,661.678437,0,train/03d56a9709a81b2e9d6afbc6647c8730.dicom
1,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,247.196368,412.000819,441.763442,830.320028,1,train/48f7ba0b0fd5ab847ee87df2aed5cfc9.dicom
2,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,613.631024,364.980443,902.238850,827.077243,1,train/48f7ba0b0fd5ab847ee87df2aed5cfc9.dicom
3,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,1008.515903,665.439667,1324.430364,1265.465119,1,train/74f8648d742bbcce79d67c967de5552d.dicom
4,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,453.015307,635.756429,724.404911,1242.142575,1,train/74f8648d742bbcce79d67c967de5552d.dicom


# Only 35 classes


In [12]:
train_df = train_df[train_df.class_id!=35].reset_index(drop = True)
train_df.head()

,image_id,rad_ID,class_name,x_min,y_min,x_max,y_max,class_id,image_path
0,03d56a9709a81b2e9d6afbc6647c8730,R3,Boot-shaped heart,378.760010,401.416645,835.143248,661.678437,0,train/03d56a9709a81b2e9d6afbc6647c8730.dicom
1,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,247.196368,412.000819,441.763442,830.320028,1,train/48f7ba0b0fd5ab847ee87df2aed5cfc9.dicom
2,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,613.631024,364.980443,902.238850,827.077243,1,train/48f7ba0b0fd5ab847ee87df2aed5cfc9.dicom
3,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,1008.515903,665.439667,1324.430364,1265.465119,1,train/74f8648d742bbcce79d67c967de5552d.dicom
4,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,453.015307,635.756429,724.404911,1242.142575,1,train/74f8648d742bbcce79d67c967de5552d.dicom


# Preprocessing

In [13]:
train_df['x_min'] = train_df.apply(lambda row: row.x_min, axis =1)
train_df['y_min'] = train_df.apply(lambda row: row.y_min, axis =1)

train_df['x_max'] = train_df.apply(lambda row: row.x_max, axis =1)
train_df['y_max'] = train_df.apply(lambda row: row.y_max, axis =1)

train_df['x_mid'] = train_df.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train_df['y_mid'] = train_df.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train_df['w'] = train_df.apply(lambda row: (row.x_max-row.x_min), axis =1)
train_df['h'] = train_df.apply(lambda row: (row.y_max-row.y_min), axis =1)

train_df['area'] = train_df['w']*train_df['h']

train_df.head()

,image_id,rad_ID,class_name,x_min,y_min,x_max,y_max,class_id,image_path,x_mid,y_mid,w,h,area
0,03d56a9709a81b2e9d6afbc6647c8730,R3,Boot-shaped heart,378.760010,401.416645,835.143248,661.678437,0,train/03d56a9709a81b2e9d6afbc6647c8730.dicom,606.951629,531.547541,456.383238,260.261792,118779.119615
1,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,247.196368,412.000819,441.763442,830.320028,1,train/48f7ba0b0fd5ab847ee87df2aed5cfc9.dicom,344.479905,621.160424,194.567074,418.319209,81391.144290
2,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,613.631024,364.980443,902.238850,827.077243,1,train/48f7ba0b0fd5ab847ee87df2aed5cfc9.dicom,757.934937,596.028843,288.607826,462.096800,133364.752925
3,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,1008.515903,665.439667,1324.430364,1265.465119,1,train/74f8648d742bbcce79d67c967de5552d.dicom,1166.473134,965.452393,315.914461,600.025452,189556.717381
4,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,453.015307,635.756429,724.404911,1242.142575,1,train/74f8648d742bbcce79d67c967de5552d.dicom,588.710109,938.949502,271.389604,606.386146,164566.896176


In [14]:
features = ['x_min', 'y_min', 'x_max', 'y_max', 'x_mid', 'y_mid', 'w', 'h', 'area']
X = train_df[features]
y = train_df['class_id']
X.shape, y.shape

((10365, 9), (10365,))

In [15]:
class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

['Boot-shaped heart',
 'Peribronchovascular interstitial opacity',
 'Reticulonodular opacity',
 'Bronchial thickening',
 'Enlarged PA',
 'Cardiomegaly',
 'Other opacity',
 'Intrathoracic digestive structure',
 'No finding',
 'Diffuse aveolar opacity',
 'Other lesion',
 'Consolidation',
 'Mediastinal shift',
 'Anterior mediastinal mass',
 'Other nodule/mass',
 'Dextro cardia',
 'Aortic enlargement',
 'Pleural effusion',
 'Stomach on the right side',
 'Atelectasis',
 'Calcification',
 'Interstitial lung disease - ILD',
 'Lung hyperinflation',
 'Egg on string sign',
 'Pulmonary fibrosis',
 'Infiltration',
 'Lung cavity',
 'Pneumothorax',
 'Edema',
 'Pleural thickening',
 'Clavicle fracture',
 'Chest wall mass',
 'Lung cyst',
 'Emphysema',
 'Bronchectasis',
 'Paraveterbral mass']

# K-fold

In [16]:
gkf  = GroupKFold(n_splits = 5)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups = train_df.image_id.tolist())):
    train_df.loc[val_idx, 'fold'] = fold
train_df.head()

,image_id,rad_ID,class_name,x_min,y_min,x_max,y_max,class_id,image_path,x_mid,y_mid,w,h,area,fold
0,03d56a9709a81b2e9d6afbc6647c8730,R3,Boot-shaped heart,378.760010,401.416645,835.143248,661.678437,0,train/03d56a9709a81b2e9d6afbc6647c8730.dicom,606.951629,531.547541,456.383238,260.261792,118779.119615,4
1,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,247.196368,412.000819,441.763442,830.320028,1,train/48f7ba0b0fd5ab847ee87df2aed5cfc9.dicom,344.479905,621.160424,194.567074,418.319209,81391.144290,2
2,48f7ba0b0fd5ab847ee87df2aed5cfc9,R3,Peribronchovascular interstitial opacity,613.631024,364.980443,902.238850,827.077243,1,train/48f7ba0b0fd5ab847ee87df2aed5cfc9.dicom,757.934937,596.028843,288.607826,462.096800,133364.752925,2
3,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,1008.515903,665.439667,1324.430364,1265.465119,1,train/74f8648d742bbcce79d67c967de5552d.dicom,1166.473134,965.452393,315.914461,600.025452,189556.717381,0
4,74f8648d742bbcce79d67c967de5552d,R3,Peribronchovascular interstitial opacity,453.015307,635.756429,724.404911,1242.142575,1,train/74f8648d742bbcce79d67c967de5552d.dicom,588.710109,938.949502,271.389604,606.386146,164566.896176,0


In [17]:
train_files = []
val_files   = []
val_files += list(train_df[train_df.fold==fold].image_path.unique())
train_files += list(train_df[train_df.fold!=fold].image_path.unique())
len(train_files), len(val_files)

(6181, 1546)

In [18]:
print(train_files)

['train/48f7ba0b0fd5ab847ee87df2aed5cfc9.dicom', 'train/74f8648d742bbcce79d67c967de5552d.dicom', 'train/e30f04846ebdf6d40eb99757f82c5a89.dicom', 'train/86ebb0c1c68ea9c4ae596a3d87833862.dicom', 'train/a9df6fd12177f1e8d6bee8dfaec87def.dicom', 'train/f00cb4808fbd215a177cb49d3d9cb024.dicom', 'train/ddad146ec2ece997adeb63f5511d1cab.dicom', 'train/8519a063841f47852cfa1da5a989ce09.dicom', 'train/43af96342f590affeaaf305437103cd7.dicom', 'train/ca2072bdff0e0fa1bf97040471f5b847.dicom', 'train/fd1837f557a8e26d938848f7e6e8aecd.dicom', 'train/948425239932f9dcddd216b5588f9f4d.dicom', 'train/e5ddff770d57ae4b9fba19a24e94a23e.dicom', 'train/620f54c7a555130927544779dfd10e1f.dicom', 'train/80c3d88c8f7f671a88eefddd84a4820c.dicom', 'train/5837db9a35a457595734d8af385a19a8.dicom', 'train/0d4149fc1dd466c852acf7682c4dcacc.dicom', 'train/2dc8746f2dab48a74f973660fcfe45de.dicom', 'train/a89771b56ab57a63423ad03cc47c1332.dicom', 'train/6802740c77a21471bbadd82ee5514f59.dicom', 'train/e627cb1ebc379b7e3aa5cf0961a4a64c

In [19]:
# # Create directories if they don't exist
# import os
# from tqdm import tqdm
# os.makedirs('D:/dataset/pcxr/yolo/labels/train', exist_ok=True)
# os.makedirs('D:/dataset/pcxr/yolo/labels/val', exist_ok=True)
# os.makedirs('D:/dataset/pcxr/yolo/images/train', exist_ok=True)
# os.makedirs('D:/dataset/pcxr/yolo/images/val', exist_ok=True)

# label_dir = 'D:/dataset/pcxr/archive/labels/'

# # Copy files for training set
# for file in tqdm(train_files):  
#     file = "D:/dataset/pcxr/images/" + file
#     shutil.copy(file, 'D:/dataset/pcxr/yolo/images/train')
# #     filename = os.path.basename(file)[:-6]
# #     shutil.copy(os.path.join(label_dir, filename+'.csv'), 'D:/dataset/pcxr/yolo/labels/train')

# # # Copy files for validation set
# # for file in tqdm(val_files):
# #     shutil.copy(file, 'D:/dataset/pcxr/yolo/images/val')
# #     filename = os.path.basename(file)[:-6]
# #     shutil.copy(os.path.join(label_dir, filename+'.txt'), 'D:/dataset/pcxr/yolo/labels/val')


# Get class name

In [20]:
class_ids, class_names = list(zip(*set(zip(train_df.class_id, train_df.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

['Boot-shaped heart',
 'Peribronchovascular interstitial opacity',
 'Reticulonodular opacity',
 'Bronchial thickening',
 'Enlarged PA',
 'Cardiomegaly',
 'Other opacity',
 'Intrathoracic digestive structure',
 'No finding',
 'Diffuse aveolar opacity',
 'Other lesion',
 'Consolidation',
 'Mediastinal shift',
 'Anterior mediastinal mass',
 'Other nodule/mass',
 'Dextro cardia',
 'Aortic enlargement',
 'Pleural effusion',
 'Stomach on the right side',
 'Atelectasis',
 'Calcification',
 'Interstitial lung disease - ILD',
 'Lung hyperinflation',
 'Egg on string sign',
 'Pulmonary fibrosis',
 'Infiltration',
 'Lung cavity',
 'Pneumothorax',
 'Edema',
 'Pleural thickening',
 'Clavicle fracture',
 'Chest wall mass',
 'Lung cyst',
 'Emphysema',
 'Bronchectasis',
 'Paraveterbral mass']

# YOLOv5


In [21]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

D:\dataset\pcxr\yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.


In [23]:
# !pip install -r requirements.txt # install dependencies
import torch
from IPython.display import Image, clear_output  # to display images
clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.2.0+cpu CPU


## Train

In [24]:
from os import listdir
from os.path import isfile, join
import yaml

cwd = 'D:/dataset/pcxr/images/'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('D:/dataset/pcxr/images/train*'):
        f.write(path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('D:/dataset/pcxr/images/test*'):
        f.write(path+'\n')

data = dict(
    train =  join( cwd , 'train.txt') ,
    val   =  join( cwd , 'val.txt' ),
    nc    = 35,
    names = classes
    )

with open(join( cwd , 'yolov5_final.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( cwd , 'yolov5_final.yaml'), 'r')
print('\nyaml:')
print(f.read())


yaml:
names:
- Boot-shaped heart
- Peribronchovascular interstitial opacity
- Reticulonodular opacity
- Bronchial thickening
- Enlarged PA
- Cardiomegaly
- Other opacity
- Intrathoracic digestive structure
- No finding
- Diffuse aveolar opacity
- Other lesion
- Consolidation
- Mediastinal shift
- Anterior mediastinal mass
- Other nodule/mass
- Dextro cardia
- Aortic enlargement
- Pleural effusion
- Stomach on the right side
- Atelectasis
- Calcification
- Interstitial lung disease - ILD
- Lung hyperinflation
- Egg on string sign
- Pulmonary fibrosis
- Infiltration
- Lung cavity
- Pneumothorax
- Edema
- Pleural thickening
- Clavicle fracture
- Chest wall mass
- Lung cyst
- Emphysema
- Bronchectasis
- Paraveterbral mass
nc: 35
train: D:/dataset/pcxr/images/train.txt
val: D:/dataset/pcxr/images/val.txt



In [25]:
# from yolov5 import utils
# display = utils.notebook_init()  # checks

In [30]:
!python train.py --img 256 --batch 8 --epochs 1 --data D:\dataset\pcxr\images\yolov5_final.yaml --weights yolov5x.pt

wandb: WARNING  wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.

wandb: Currently logged in as: vivek-garg1407 (vinpcxr). Use `wandb login --relogin` to force relogin
train: weights=yolov5x.pt, cfg=, data=D:\dataset\pcxr\images\yolov5_final.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=1, batch_size=8, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github:  YOLOv5 is out of date by 

In [32]:
# # !zip -r /content/yolov5.zip /content/yolov5

# from google.colab import files
# files.download("/content/yolov5-2.zip")

ModuleNotFoundError: No module named 'google.colab'

# Dừng nha

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (15, 15))
plt.imshow(plt.imread('runs/train/exp/train_batch0.jpg'))
plt.savefig('fig1.png')
# plt.figure(figsize = (15, 15))
# plt.imshow(plt.imread('runs/train/exp2/train_batch1.jpg'))

# plt.figure(figsize = (15, 15))
# plt.imshow(plt.imread('runs/train/exp2/train_batch2.jpg'))

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/confusion_matrix.png'));

# Inference


In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --imgsz 256 \
--conf 0.15 \
--iou 0.5 \
--source '/content/drive/MyDrive/COURSES/CS431/CXR-Abnormalities-Detection/data/yolo/images/val/' \
--exist-ok

## Test

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/COURSES/CS431/CXR-Abnormalities-Detection/data/test.csv')
test_df.head()

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt \
--img 256 \
--conf 0.01 \
--iou 0.4 \
--source '/content/drive/MyDrive/COURSES/CS431/CXR-Abnormalities-Detection/data/test/' \
--save-txt --save-conf --exist-ok

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
image_ids = []
PredictionStrings = []

for file_path in tqdm(glob('runs/detect/exp/labels/*txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

In [ ]:
pred_df = pd.DataFrame({'image_id':image_ids,
                        'PredictionString':PredictionStrings})
sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
sub_df = sub_df[['image_id', 'PredictionString']]
sub_df.to_csv('submission.csv',index = False)
sub_df.tail()

In [ ]:
# !zip -r /content/yolov5-2.zip /content/yolov5

# from google.colab import files
# files.download("/content/yolov5.zip")